## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-11-05-11-08 +0000,nypost,Joe Rogan blasts former podcast guest who trie...,https://nypost.com/2026/02/11/us-news/joe-roga...
1,2026-02-11-05-10-00 +0000,wsj,Nine Killed and at Least 25 Injured in Canada ...,https://www.wsj.com/world/americas/nine-killed...
2,2026-02-11-05-07-35 +0000,nyt,The Bangladesh Elections Are on Thursday. Here...,https://www.nytimes.com/2026/02/11/world/asia/...
3,2026-02-11-05-01-10 +0000,nyt,"Away From Pomp of Olympics, Homeless Shiver on...",https://www.nytimes.com/2026/02/11/world/europ...
4,2026-02-11-05-01-06 +0000,nyt,Trump Wants to Revive Shipping. Investors Are ...,https://www.nytimes.com/2026/02/11/business/tr...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2433/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
31,trump,42
10,epstein,19
270,new,17
63,guthrie,13
53,ice,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
23,2026-02-11-03-37-00 +0000,wsj,House lawmakers rejected an attempt by Speaker...,https://www.wsj.com/politics/policy/house-reje...,142
174,2026-02-10-20-53-18 +0000,nyt,Former Palm Beach Police Chief Said Trump Told...,https://www.nytimes.com/2026/02/10/us/politics...,96
177,2026-02-10-20-48-27 +0000,bbc,Ex-police chief said Trump told him in 2006 'e...,https://www.bbc.com/news/articles/cx2407jrn83o...,92
20,2026-02-11-03-51-40 +0000,bbc,Under-fire Trump commerce secretary confirms h...,https://www.bbc.com/news/articles/cwyx9vq0ne0o...,92
230,2026-02-10-18-37-00 +0000,wsj,"Republican Sen. Susan Collins, the Maine centr...",https://www.wsj.com/politics/policy/susan-coll...,87


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
23,142,2026-02-11-03-37-00 +0000,wsj,House lawmakers rejected an attempt by Speaker...,https://www.wsj.com/politics/policy/house-reje...
38,70,2026-02-11-02-45-29 +0000,nyt,New Photos Show Masked Person on Nancy Guthrie...,https://www.nytimes.com/2026/02/10/us/nancy-gu...
0,55,2026-02-11-05-11-08 +0000,nypost,Joe Rogan blasts former podcast guest who trie...,https://nypost.com/2026/02/11/us-news/joe-roga...
273,46,2026-02-10-16-15-01 +0000,nypost,Gavin Newsom ripped by LA’s top fed prosecuto...,https://nypost.com/2026/02/10/us-news/gavin-ne...
170,38,2026-02-10-21-15-41 +0000,nypost,LI parents who tragically lost hockey star son...,https://nypost.com/2026/02/10/us-news/li-paren...
117,37,2026-02-10-23-03-05 +0000,latimes,$600 million in Trump administration health cu...,https://www.latimes.com/politics/story/2026-02...
82,36,2026-02-11-00-12-21 +0000,latimes,LAPD to train their body cameras on immigratio...,https://www.latimes.com/california/story/2026-...
307,34,2026-02-10-13-00-00 +0000,latimes,He worked for and romanced Arcadia councilwoma...,https://www.latimes.com/california/story/2026-...
276,34,2026-02-10-16-07-54 +0000,nypost,Nervous Netanyahu pushes up trip to DC for hig...,https://nypost.com/2026/02/10/us-news/nervous-...
358,33,2026-02-10-08-18-20 +0000,nypost,Fiend sobs over missing Grand Theft Auto video...,https://nypost.com/2026/02/10/world-news/man-s...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
